In [ ]:
import pandas as pd 
import numpy as np 
import scipy as sp
import string
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
import xgboost as xgb
from sklearn.decomposition import TruncatedSVD

In [ ]:
file = "train_tweets.txt"
stop = stopwords.words('english')
nltk.download('wordnet')
# file = "data/train_tweets.txt"
temp = []
with open(file,encoding="UTF-8") as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row)

In [ ]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []
# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()
# def lemmatize_text(text):
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [ ]:
# tw['Tweet'].str.replace('[{}]'.format(string.punctuation), '')
# tw['Tweet'].apply(lambda x: [item for item in x if item not in stop])
# tw['Tweet'].apply(lemmatize_text)

In [ ]:
# test=tw.groupby('User').count()

In [ ]:
# max_user=test.nlargest(7000, 'Tweet')
# max_user

In [ ]:
import html
tw["Tweet"] = tw['Tweet'].apply(lambda x: html.unescape(x))

In [ ]:
X=tw['Tweet']
y=tw['User']
X[7009]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=1234)

In [ ]:
# Accuracy with 0.308
# word_vectorizer = TfidfVectorizer(stop_words = 'english', ngram_range = (1,5),sublinear_tf=True,min_df = 2,max_features=60000)
# char_vectorizer = CountVectorizer(analyzer='char', stop_words='english', ngram_range=(2,5), max_features=60000)
#0.32
word_vectorizer = TfidfVectorizer(stop_words = 'english', ngram_range = (1,6),sublinear_tf=True,min_df = 2,max_features=60000)
char_vectorizer = TfidfVectorizer(analyzer='char', stop_words='english', ngram_range=(2,6), max_features=60000)
special_vectorizer = TfidfVectorizer(token_pattern='(?u)(@\w+)|(#\w+)|(!+)|(\?+)|(:\))|(:D)|(:o)|(:O)|(\.+)', stop_words='english', max_features=20000)

In [ ]:
from scipy.sparse import coo_matrix, hstack
word_vectorizer.fit(pd.Series(tw['Tweet']))
char_vectorizer.fit(pd.Series(tw['Tweet']))
special_vectorizer.fit(pd.Series(tw['Tweet']))
X_train_dtm_w = word_vectorizer.transform(X_train)
X_train_dtm_c = char_vectorizer.transform(X_train)
X_train_dtm_special=special_vectorizer.transform(X_train)
X_train_dtm = hstack([X_train_dtm_c, X_train_dtm_w,X_train_dtm_special])



x_test_count_c=char_vectorizer.transform(X_test)
x_test_special=special_vectorizer.transform(X_test)
x_test_count_w=word_vectorizer.transform(X_test)
X_test_dtm = hstack([x_test_count_c, x_test_count_w,x_test_special])

In [ ]:
# %%time
model=LinearSVC(max_iter=10000)
model.fit(X_train_dtm,y_train)

In [ ]:
# model
y_pred=model.predict(X_test_dtm)

In [ ]:
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
def prepare_test_data(vec_w,vec_c,vec_special):
    file1 = "test_tweets_unlabeled.txt"
    with open(file1,encoding="UTF-8") as data:
        temp = [line for line in data]    
    unlabel = pd.Series(temp)
    unlabel_dtm_w = vec_w.transform(unlabel)
    unlabel_dtm_c = vec_c.transform(unlabel)
    unlabel_dtm_special=vec_special.transform(unlabel)
    unlabel_dtm= hstack([unlabel_dtm_c,unlabel_dtm_w,unlabel_dtm_special])
    return unlabel_dtm
    
def submission_file(data):
    import csv
    with open('predicted.csv', 'w') as writeFile:
        writer = csv.writer(writeFile,lineterminator='\n')
        writer.writerow(['Id','Predicted'])
        for count,predicted in enumerate(data):
            writer.writerow([count+1,predicted])

In [ ]:
unlabel_dtm = prepare_test_data(word_vectorizer,char_vectorizer,special_vectorizer) 
unlabel_pred = model.predict(unlabel_dtm)
submission_file(unlabel_pred)